In [1]:
from phi.torch.flow import *

from src.eigenfluid import Eigenfluid
from src.shapes import *

# Helper functions for visualization
from src.visu import *

import time

In [2]:
# Number of basis fields
N = 4**2
# Domain sizes
DOMAIN = Box(x=math.PI, y=math.PI)
# NOT for sampling points, but for reconstructing the velocity grid
SAMPLING_SIZE = 32

start_time = time.time()
eigenfluid = Eigenfluid(N, DOMAIN, SAMPLING_SIZE, init_w='zero')
end_time = time.time()
print("Eigenfluid initialized in {} seconds".format(end_time-start_time))

Eigenfluid initialized in 9.312891960144043 seconds


# Data

In [3]:
# Get O overlapping, and U non-necessarily overlapping ('unique') sample points
O = 0
U = 0
N_S = O+U+5 # (5 trivial points)

BATCH_SIZE = 200
VISU_NR = 5

def get_points_for_shapes(shape_0, shape_t):
    p_0 = shape_0.get_trivial_points()
    p_t = shape_target.get_trivial_points()
    if O != 0:
        sampler_union = ShapeSampler(shape_0, shape_t, N=O, h1=2, h2=7)
        p_0 = math.concat((p_0, sampler_union.p), instance('i'))
        p_t = math.concat((p_t, sampler_union.p), instance('i'))
    if U != 0:
        sampler_0 = ShapeSampler(shape_0, N=U, h1=3, h2=11)
        sampler_t = ShapeSampler(shape_t, N=U, h1=3, h2=11)
        # Concatenate both the union and non-union points
        p_0 = math.concat((p_0, sampler_0.p), instance('i'))
        p_t = math.concat((p_t, sampler_t.p), instance('i'))
    
    p_0 = shape_0.create_points(p_0)
    p_t = shape_target.create_points(p_t)
    
    return (p_0, p_t)

p_0_list = []
p_t_list = []
for i in range(BATCH_SIZE):
    # Circle / Square / Triangle
    Shape_0 = [Circle, Square, Triangle][np.random.randint(3)]
    Shape_target = [Circle, Square, Triangle][np.random.randint(3)]
    # Random positions [0,2.1] x [0,2.1]
    shape_0 = Shape_0(pos=(np.random.rand()*2.1, np.random.rand()*2.1))
    shape_target = Shape_target(pos=(np.random.rand()*2.1, np.random.rand()*2.1))
    
    p_0, p_t = get_points_for_shapes(shape_0, shape_target)
    p_0_list.append(p_0)
    p_t_list.append(p_t)

    
p_0 = math.stack(p_0_list, batch(batch=BATCH_SIZE))
p_t = math.stack(p_t_list, batch(batch=BATCH_SIZE))

# Generate transition data
STEPS = 16 #p_0 is at time step 1
p_trajectory = math.tensor([p_0+(p_t - p_0)/STEPS*t for t in range(STEPS)], batch('time') & p_0.shape)

plot({
    "Blue: start shape\nRed: target shape\nGreen: Linear Trajectory": vis.overlay(
        PointCloud(p_0.batch[:VISU_NR], bounds=DOMAIN),
        PointCloud(p_t.batch[:VISU_NR], bounds=DOMAIN, color="red"),
        PointCloud(p_trajectory.batch[:VISU_NR], bounds=DOMAIN, color="green"),
     )}, 
     animate='time',
     size=(12,4))

<Figure size 432x288 with 0 Axes>

# Loss Function

In [4]:
STEPS = 16 # Already defined during data generation
DT    = 0.2
VISCOSITY = 0.0

def to_model_flat(p_0, p_t, w):
    return math.concat([
        p_0['x'], p_0['y'],
        p_t['x'], p_t['y'],
        math.rename_dims(w, 'k', instance('i'))
    ], instance('i'))

def from_model_flat(net_output):
    return math.rename_dims(tensor(
        net_output,
        batch('batch') & channel('channels') & channel('vector')
    ).channels[0], 'vector', instance('k'))


@math.jit_compile
def loss_function(p_0, p_t, w):
    loss = 0
    p_est = p_0.time[0]
    p_est_traj = [p_est]
    for t in range(STEPS-1):
        # Feed data into NN to get CFE prediction
        model_input = to_model_flat(p_est, p_t.time[t], w)
        model_output = net(model_input.native('batch,channels,i'))
        f_pred = from_model_flat(model_output)
        #print("t={}, f_pred={}".format(t, f_pred))
        w += f_pred
        # Step simulation
        w = eigenfluid.step_w_euler(w, DT, viscosity=VISCOSITY)
        # Get velocities at p_0
        v_phi = eigenfluid.phi_template(w, eigenfluid.N, eigenfluid.basis_fields)
        p_velocities = v_phi(p_est)
        # Midpoint
        p_velocities = v_phi(p_est + p_velocities*DT/2)
        # Euler step for points
        p_est += p_velocities * DT
        p_est_traj.append(p_est)
        loss += math.l2_loss(p_est - p_t.time[t])
    return loss, p_est_traj

# Network and Training

In [5]:
# Input: [p_current, p_target, current_w]
# -> Input dimension = [O+U, O+U, N]
# Output = [N] force coefficients for current time step
#net = dense_net(2*2*(O+U)+N, N, [256, 128, 64, 32], batch_norm=True)
net = dense_net(N_S*2*2+N, N, [256, 128, 64, 32])
optimizer = adam(net)

print(net)
print("Number of parameters: {}".format(parameter_count(net)))
#print()
#print("Optimizer:{}".format(optimizer))

DenseNet(
  (linear0): Linear(in_features=36, out_features=256, bias=True)
  (linear1): Linear(in_features=256, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=64, bias=True)
  (linear3): Linear(in_features=64, out_features=32, bias=True)
  (linear4): Linear(in_features=32, out_features=16, bias=True)
)
Number of parameters: 53232


### Visualize Untrained CFE NN (first 5)

In [6]:
## Test the loss function and the NN
loss, cfe_traj = loss_function(
                    p_trajectory.time[0:STEPS-1].batch[:VISU_NR],
                    p_trajectory.time[1:STEPS].batch[:VISU_NR],
                    math.stack([eigenfluid.w]*VISU_NR, batch('batch'))
                )
print("Loss = {}".format(loss))
cfe_traj = math.stack(cfe_traj, batch('time'))
plot(PointCloud(cfe_traj, bounds=DOMAIN), animate='time')

/Users/bobarna/codes/phiflow/phi/torch/_torch_backend.py:480: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return tuple([int(s) for s in tensor.shape])
/Users/bobarna/codes/phiflow/phi/torch/_torch_backend.py:54: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  tensor = torch.from_numpy(x)
/Users/bobarna/codes/phiflow/phi/math/_tensors.py:662: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might 

Loss = (61.662, 77.197, 19.162, 20.910, 38.750) along batchᵇ


<Figure size 432x288 with 0 Axes>

## Train

In [7]:
for epoch in range(1000):
    w_optim = math.stack([eigenfluid.w]*BATCH_SIZE, batch('batch'))
    loss, p_est_traj = update_weights(net, optimizer, loss_function, 
                   p_trajectory.time[0:STEPS-1], 
                   p_trajectory.time[1:STEPS], 
                   w_optim)
    if epoch%10 == 0 or epoch < 5:
        print("Optimization step {}, loss: {}".format(epoch, loss))

/Users/bobarna/codes/phiflow/phi/torch/_torch_backend.py:480: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return tuple([int(s) for s in tensor.shape])
/Users/bobarna/codes/phiflow/phi/torch/_torch_backend.py:54: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  tensor = torch.from_numpy(x)
/Users/bobarna/codes/phiflow/phi/math/_tensors.py:662: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might 

Optimization step 0, loss: (batchᵇ=200) 30.903 ± 13.976 (9e+00...8e+01)
Optimization step 1, loss: (batchᵇ=200) 30.420 ± 14.038 (9e+00...7e+01)
Optimization step 2, loss: (batchᵇ=200) 29.722 ± 13.971 (8e+00...8e+01)
Optimization step 3, loss: (batchᵇ=200) 30.516 ± 14.168 (9e+00...8e+01)
Optimization step 4, loss: (batchᵇ=200) 30.723 ± 14.227 (9e+00...8e+01)
Optimization step 10, loss: (batchᵇ=200) 28.793 ± 13.333 (8e+00...7e+01)
Optimization step 20, loss: (batchᵇ=200) 25.847 ± 12.745 (7e+00...7e+01)
Optimization step 30, loss: (batchᵇ=200) 22.276 ± 11.372 (6e+00...7e+01)
Optimization step 40, loss: (batchᵇ=200) 18.995 ± 11.482 (3e+00...7e+01)
Optimization step 50, loss: (batchᵇ=200) 15.197 ± 10.336 (1e+00...5e+01)
Optimization step 60, loss: (batchᵇ=200) 12.283 ± 8.398 (2e+00...5e+01)
Optimization step 70, loss: (batchᵇ=200) 10.381 ± 7.406 (9e-01...4e+01)
Optimization step 80, loss: (batchᵇ=200) 9.262 ± 6.889 (1e+00...4e+01)
Optimization step 90, loss: (batchᵇ=200) 8.221 ± 6.216 (7e-0

KeyboardInterrupt: 

## Visualize Results (first 5)

In [8]:
loss, cfe_traj = loss_function(
                    p_trajectory.time[0:STEPS-1].batch[15:15+VISU_NR],
                    p_trajectory.time[1:STEPS].batch[15:15+VISU_NR],
                    math.stack([eigenfluid.w]*VISU_NR, batch('batch'))
                )
print("Loss = {}".format(loss))
cfe_traj = math.stack(cfe_traj, batch('time'))
plot(
    vis.overlay(
        PointCloud(cfe_traj, bounds=DOMAIN),
        PointCloud(p_trajectory.time[STEPS-1].batch[15:15+VISU_NR], bounds=DOMAIN, color='red')
    ),
    animate='time'
)

Loss = (2.314, 2.446, 1.543, 2.707, 3.443) along batchᵇ


<Figure size 432x288 with 0 Axes>

# Test Generalization of NN on Unseen Data

## Generate Test Data

In [9]:
TEST_SIZE = 10

# Generate test data
p_0_test_list = []
p_t_test_list = []
for i in range(TEST_SIZE):
    # Circle / Square / Triangle
    Shape_0 = [Circle, Square, Triangle][np.random.randint(3)]
    Shape_target = [Circle, Square, Triangle][np.random.randint(3)]
    # Random positions [0,2] x [0,2]
    shape_0 = Shape_0(pos=(np.random.rand()*2, np.random.rand()*2))
    shape_target = Shape_target(pos=(np.random.rand()*2, np.random.rand()*2))
    
    p_0, p_t = get_points_for_shapes(shape_0, shape_target)
    p_0_test_list.append(p_0)
    p_t_test_list.append(p_t)

    
p_0_test = math.stack(p_0_test_list, batch(batch=TEST_SIZE))
p_t_test = math.stack(p_t_test_list, batch(batch=TEST_SIZE))

# Generate transition data
p_test_trajectory = math.tensor([p_0_test+(p_t_test - p_0_test)/STEPS*t for t in range(STEPS)], batch('time') & p_0_test.shape)

## Run Tests

In [10]:
loss, test_traj = loss_function(
                    p_test_trajectory.time[0:STEPS-1],
                    p_test_trajectory.time[1:STEPS],
                    math.stack([eigenfluid.w]*TEST_SIZE, batch('batch'))
                )
print("Loss = {}".format(loss))
test_traj = math.stack(test_traj, batch('time'))
plot(
    vis.overlay(
        PointCloud(test_traj.batch[:VISU_NR], bounds=DOMAIN),
        PointCloud(p_t_test.batch[:VISU_NR], bounds=DOMAIN, color="red"),
    ),
    animate='time'
)

/Users/bobarna/codes/phiflow/phi/torch/_torch_backend.py:480: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return tuple([int(s) for s in tensor.shape])
/Users/bobarna/codes/phiflow/phi/torch/_torch_backend.py:54: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  tensor = torch.from_numpy(x)
/Users/bobarna/codes/phiflow/phi/math/_tensors.py:662: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might 

Loss = (batchᵇ=10) 3.267 ± 3.689 (6e-01...1e+01)


<Figure size 432x288 with 0 Axes>